In [ ]:
### 1. Obtain the dataset

In [ ]:
### 2. Extract the .csv file from the zipped archive, and load it into a Dask dataframe.

In [3]:
from dask.distributed import Client
import dask.dataframe as dd

# dask client with 4 workers
clt = Client(n_workers=4, serializers=['pickle'])

# COVID-19 dataset 
df = dd.read_csv("us_simplified_csv.csv")

# census data
census = dd.read_csv("2019_Census_US_Population_Data_By_State_Lat_Long.csv")

# merging data
us_simple = dd.merge(df, census, left_on='Province/State', right_on='STATE', how='left')

# merged data saved
us_simple.to_csv('us_simple.csv', index=False)

# reading merged
data = dd.read_csv('us_simple.csv')

# datafram
data.compute()


C:\Users\cmnea\anaconda3\Lib\site-packages\distributed\node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 58014 instead
  warnings.warn(
C:\Users\cmnea\anaconda3\Lib\contextlib.py:144: UserWarning: Creating scratch directories is taking a surprisingly long time. (1.25s) This is often due to running workers on a network file system. Consider specifying a local-directory to point workers to write scratch data to a local disk.
  next(self.gen)
2024-03-03 21:08:17,653 - distributed.protocol.core - CRITICAL - Failed to deserialize
Traceback (most recent call last):
  File "C:\Users\cmnea\anaconda3\Lib\site-packages\distributed\protocol\core.py", line 160, in loads
    return msgpack.loads(
           ^^^^^^^^^^^^^^
  File "C:\Users\cmnea\anaconda3\Lib\site-packages\msgpack\fallback.py", line 136, in unpackb
    raise ExtraData(ret, unpacker._get_extradata())
msgpack.exceptions.ExtraData: unpack(b) received extra

CancelledError: _write_csv-9887ea7e-2b80-4639-aa11-099a1d606bb0

In [ ]:
### 3. Using this dataframe as the starting point, perform the following steps:
    # 1. Create a new dataframe object that consists of samples (i.e., rows) corresponding to states in the US.
    # 2. During the time period 2020-Jan-01 to 2021-Feb-28, rank states in terms of their per-capita mortality? 
        # Compute per-capita mortality during a specific period as the ratio of total deaths during that time period, 
        # to the average population of the state (compute the average population during the time period).
    # 3. During the same time period, compute the case fatality rate (CFR) per month, using one of the approaches defined in this 
        # scientific brief from the World Health Organizationity. This computation should yield an array of dimensions 50 (states) X 14 (months). 
        # State the assumptions you are making in computing this metric.
    # 4. Using this matrix as input, compute the ranking of states, on how the CFR rate changed over time. This computation will involve an aggregation 
        # of of month-to-month changes in CFR. Note that some of these month-to-month changes can be positive (CFR increases from a previous month to the 
        # current month) or negative (CFR decrease from a previous month to the current month), or zero (no change in CFR). You need to aggregate these 
        # individual changes, across all time periods

In [7]:
# US states
us_states = [
    'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado',
    'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho',
    'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana',
    'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
    'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
    'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
    'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
    'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota',
    'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
    'West Virginia', 'Wisconsin', 'Wyoming'
]

# filtering
us_data = us_simple[us_simple['Province/State'].isin(us_states)]


us_data



,Date,Admin2,Province/State,Confirmed,Deaths,Country/Region,STATE,POPESTIMATE2019,lat,long
npartitions=1,,,,,,,,,,
,string,string,string,int64,int64,string,string,int64,float64,float64
,...,...,...,...,...,...,...,...,...,...


In [11]:
import dask.dataframe as dd
from dask.distributed import Client

# starting a Dask client
client = Client(n_workers=4)

# reading data
us_simple = dd.read_csv('us_simplified_csv.csv')

# datetime format coversion
us_simple['Date'] = dd.to_datetime(us_simple['Date'])

# flitering time period data
filtered_data = us_simple[(us_simple['Date'] >= '2020-01-01') & (us_simple['Date'] <= '2021-02-28')]

# grouping
total_deaths = filtered_data.groupby('Province/State')['Deaths'].sum().compute()

# grouping
average_population = filtered_data.groupby('Province/State')['Population'].mean().compute()

# calculating
per_capita_mortality = total_deaths / average_population

# ranking states
ranked_states = per_capita_mortality.sort_values(ascending=False)

# ranked states
print(ranked_states)


C:\Users\cmnea\anaconda3\Lib\site-packages\distributed\node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 58378 instead
  warnings.warn(
C:\Users\cmnea\anaconda3\Lib\contextlib.py:144: UserWarning: Creating scratch directories is taking a surprisingly long time. (2.74s) This is often due to running workers on a network file system. Consider specifying a local-directory to point workers to write scratch data to a local disk.
  next(self.gen)
2024-03-03 21:18:14,950 - distributed.protocol.core - CRITICAL - Failed to deserialize
Traceback (most recent call last):
  File "C:\Users\cmnea\anaconda3\Lib\site-packages\distributed\protocol\core.py", line 160, in loads
    return msgpack.loads(
           ^^^^^^^^^^^^^^
  File "C:\Users\cmnea\anaconda3\Lib\site-packages\msgpack\fallback.py", line 136, in unpackb
    raise ExtraData(ret, unpacker._get_extradata())
msgpack.exceptions.ExtraData: unpack(b) received extra

CancelledError: ('series-groupby-sum-agg-00b1c3f7c6f2d7744c510d99577062a5', 0)

In [18]:
import dask.dataframe as dd
from datetime import datetime

us_simple = dd.read_csv('us_simplified_csv.csv')

# datetime format
us_simple['Date'] = dd.to_datetime(us_simple['Date'])

# filtering dates
start_date = datetime(2020, 1, 1)
end_date = datetime(2021, 2, 28)
filtered_data = us_simple[(us_simple['Date'] >= start_date) & (us_simple['Date'] <= end_date)]

# getting month and year
filtered_data['Month'] = filtered_data['Date'].dt.to_period('M')
filtered_data['Month'] = filtered_data['Month'].astype('category')  # Convert to category dtype
filtered_data['Month'] = filtered_data['Month'].cat.as_known()

# grouping
monthly_data = filtered_data.groupby(['Province/State', 'Month']).agg({'Confirmed': 'sum', 'Deaths': 'sum'}).reset_index()

# calculating
monthly_data['CFR'] = (monthly_data['Deaths'] / monthly_data['Confirmed']) * 100

# pivot the data to create an array of dimensions 50 (states) x 14 (months)
cfr_array = monthly_data.pivot_table(index='Province/State', columns='Month', values='CFR')

# array of CFR values
print(cfr_array.compute())


2024-03-03 21:28:26,568 - distributed.protocol.core - CRITICAL - Failed to deserialize
Traceback (most recent call last):
  File "C:\Users\cmnea\anaconda3\Lib\site-packages\distributed\protocol\core.py", line 160, in loads
    return msgpack.loads(
           ^^^^^^^^^^^^^^
  File "C:\Users\cmnea\anaconda3\Lib\site-packages\msgpack\fallback.py", line 136, in unpackb
    raise ExtraData(ret, unpacker._get_extradata())
msgpack.exceptions.ExtraData: unpack(b) received extra data.
2024-03-03 21:28:28,019 - distributed.core - ERROR - Exception while handling op register-client
Traceback (most recent call last):
  File "C:\Users\cmnea\anaconda3\Lib\site-packages\distributed\core.py", line 968, in _handle_comm
    result = await result
             ^^^^^^^^^^^^
  File "C:\Users\cmnea\anaconda3\Lib\site-packages\distributed\scheduler.py", line 5532, in add_client
    await self.handle_stream(comm=comm, extra={"client": client})
  File "C:\Users\cmnea\anaconda3\Lib\site-packages\distributed\cor

CancelledError: ('unique-agg-54210c1cef70c2670ac87c511acc88de', 0)